Import

In [1]:
import pandas as pd
import class_def as cd
import re
import random
import openpyxl
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment
from openpyxl.styles import Color, PatternFill, Font, Border
from openpyxl.styles.borders import Border, Side

def printErrorMsg(fileName):
    print(fileName)
    print('Press any key to continue ...')
    input()
    exit()

print('Reading data ...')

try:
    df= pd.read_excel('Input/Specific Examer.xlsx')
except:
    printErrorMsg('Can\'t find file \'Specific Examer.xlsx\'!')

MAIN_EXAMER_OF_ENG_SPEAKING = [x for x in df['English Speaking\n主考官'].tolist() if x == x]
ORAL_EXAMER_OF_ENG_SPEAKING = [x for x in df['English Speaking\nOral 考官'].tolist() if x == x]
MAIN_EXAMER_OF_ENG_LISTENING = [x for x in df['English Listening\n主考官'].tolist() if x == x]
MAIN_EXAMER_OF_CHIN_SPEAKING = [x for x in df['中文説話\n主考官'].tolist() if x == x]
ORAL_EXAMER_OF_CHIN_SPEAKING = [x for x in df['中文説話\nOral 考官'].tolist() if x == x]
MAIN_EXAMER_OF_CHIN_LISTENING = [x for x in df['中文聆聽\n主考官'].tolist() if x == x]
MAIN_EXAMER_OF_PTH = [x for x in df['普通話\n主考官'].tolist() if x == x]
MAIN_EXAMER_OF_VA = [x for x in df['VA\n主考官'].tolist() if x == x]

tmp = {}
for examer in MAIN_EXAMER_OF_VA:
    examer = examer.replace(' ', '')
    tmp[int(examer[0])] = examer[2:]
MAIN_EXAMER_OF_VA = tmp

CANT_BE_EXAMER = [x for x in (df['不能監考\n(校長)'].tolist() + df['外籍老師'].tolist()) if x == x]

TA_DATA = []
for ta in [x for x in df['TA'].tolist() if x == x]:
    TA_DATA.append(cd.TA(name=ta))

Reading data ...


Reading exam timetable data

In [2]:
try:
    df= pd.read_excel('Input/Exam Timetable.xlsx', skiprows=[0], usecols=lambda x: 'Unnamed' not in x)
except:
    printErrorMsg('Can\'t find file \'Exam Timetable.xlsx\'!')

ET_DATA = []
for date in df.columns:
    if date[-2:-1] in ['一', '二', '三', '四', '五', '六', '日']:
        ET_DATA.append(cd.exam(examDate=date))
    else:
        print('Exam Timetable: Date Formate Error!')
    
for exam in ET_DATA:
    exam.subjects = []
    exam.noExam = []
    listedColum = df[exam.examDate].tolist()
    listedColum.insert(0, exam.examDate)
    form = 0
    for i in range(len(listedColum)-1):
        if listedColum[i] == exam.examDate:
            form += 1
            if listedColum[i+1] == '上課':
                exam.noExam.append(form)
    listedColum = [x for x in listedColum if x == x] #remove nan
    listedColum = list(filter(lambda i: i != '上課', listedColum)) #remove '上課'
    i = 0
    form = 0
    while i < len(listedColum):
        if listedColum[i] == exam.examDate:
            form += 1
            i += 1
        else:
            exam.subjects.append(cd.subject(name = listedColum[i], timeLimit = listedColum[i+1], room = listedColum[i+2], period = listedColum[i+3], form=form))
            i += 4

Reading teacher time table data

In [3]:
def findForm(lessonName):
    if re.match('[1-9]', lessonName[0]):
        return lessonName[0]
    else:
       return 'all'
    
def getClass(lessonName):
    listedClass = []
    pattern = re.compile(r'[1-9]+[0-9]+[0-9]')
    listedClass = pattern.findall(lessonName)
    return listedClass

try:
    sheets = pd.ExcelFile('Input/Teacher Timetable.xlsx')
    timeSlot = pd.read_excel('Input/Teacher Timetable.xlsx', skiprows=[0,1,3,6,10,13,16])['Unnamed: 0'].tolist()
except:
    printErrorMsg('Can\'t find file \'Teacher Timetable.xlsx\'!')

TT_DATA = []
dateDict = {'Mon' : '一', 'Tue' : '二', 'Wed' : '三', 'Thu' : '四', 'Fri' : '五'}
for sheetName in sheets.sheet_names:
    if sheetName not in CANT_BE_EXAMER:
        TT_DATA.append(cd.teacher(sheetName))
for teacher in TT_DATA:
    teacher.lessons = {}
    teacher.totalTime = 0
    teacher.classes = []
    teacher.exams = {}
    df = pd.read_excel('Input/Teacher Timetable.xlsx', skiprows=[0,1,3,6,10,13,16], sheet_name=teacher.name, usecols=lambda x: 'Unnamed' not in x)
    for date in df.columns:
        noLesson = df[date].isnull().tolist()
        listedColum = df[date].tolist()
        teacher.lessons[dateDict[date]] = {}
        for i in range(len(noLesson)):
            if noLesson[i] is False:
                teacher.lessons[dateDict[date]][timeSlot[i]] = findForm(listedColum[i])
                teacher.classes += getClass(listedColum[i])
    teacher.classes = [*set(teacher.classes)]

for teacher in TT_DATA:
    teacher.totalTime = 0
    for exam in ET_DATA:
        for needLessonForms in exam.noExam:
            for key in teacher.lessons[exam.examDate[-2:-1]]:
                if teacher.lessons[exam.examDate[-2:-1]][key] == str(needLessonForms) or teacher.lessons[exam.examDate[-2:-1]][key] == 'all':
                    teacher.totalTime += 35

Distributing teachers to exams

In [4]:
def checkTime(examTime, lessonTime):
    time1 = []
    time2 = []
    numPattern = re.compile(r'\d+')
    time1.append(int(numPattern.findall(examTime)[0])*60+int(numPattern.findall(examTime)[1]))
    time1.append((int(numPattern.findall(examTime)[-2]) + (12 if (re.search( r'p', examTime, re.I) and len(numPattern.findall(examTime)[-2]) < 2) else 0))*60+int(numPattern.findall(examTime)[-1]))
    time2.append(int(numPattern.findall(lessonTime)[0])*60+int(numPattern.findall(lessonTime)[1]))
    time2.append((int(numPattern.findall(lessonTime)[-2]) + (12 if (re.search( r'p', lessonTime, re.I) and len(numPattern.findall(lessonTime)[-2]) < 2) else 0))*60+int(numPattern.findall(lessonTime)[-1]))
    if (time1[0] > time2[1]) or (time1[1] < time2[0]):
        return False
    else:
        return True


def findAvalibleTeachers(subject, exam, specificExamer=None):
    avalibleTeachersList = []
    teacherData = TT_DATA
    if specificExamer != None:
        teacherData = []
        for teacherNames in specificExamer:
            teacherData.append(findParentObj(TT_DATA, teacherNames))
    for teacher in teacherData:
        isBussy = False
        isTheirTeacher = False
        if len(exam.noExam) > 0:
            for value in teacher.lessons[exam.examDate[-2:-1]].values(): 
                if value in exam.noExam or value == 'all':
                    for lessonTime in [key for key in teacher.lessons[exam.examDate[-2:-1]] if (teacher.lessons[exam.examDate[-2:-1]][key] == value or teacher.lessons[exam.examDate[-2:-1]][key] == 'all')]:
                        if not isBussy:
                            isBussy = checkTime(subject.period, lessonTime)
                        else:
                            break
                if isBussy:
                    break
        if exam.examDate in [key for key in teacher.exams]:
            for examTime in teacher.exams[exam.examDate]:
                if not isBussy:
                    isBussy = checkTime(subject.period, examTime)
                else:
                    break
        # if 'HALL' not in subject.room:
        #     for teachedClass in teacher.classes:
        #         if teachedClass in subject.room:
        #             isTheirTeacher = True
        if not isBussy and not isTheirTeacher:
            avalibleTeachersList.append(teacher)
            
    avalibleTeachersList.sort(key=lambda x: x.totalTime, reverse=False)
    return avalibleTeachersList[0]

def findParentObj(data, name):
    return data[list(map(lambda x : x.name == name, data)).index(True)]

def appendTeachers(i, subject, exam, avalibleTeacher):
    if subject.teachers[i] != '':
        return
    subject.teachers[i] = avalibleTeacher.name
    avalibleTeacher.totalTime += subject.timeLimit
    if exam.examDate not in [key for key in avalibleTeacher.exams]:
        avalibleTeacher.exams[exam.examDate] = []
    avalibleTeacher.exams[exam.examDate].append(subject.period)

def appendTA(i, subject):
    avalibleTAList = TA_DATA
    avalibleTAList.sort(key=lambda x: x.totalTime, reverse=False)
    avalibleTA = avalibleTAList[0]
    subject.teachers[i] = avalibleTA.name
    avalibleTA.totalTime += subject.timeLimit

print('Processing ...')

for exam in ET_DATA:
    for subject in exam.subjects:
        if 'peaking' in subject.name:
            appendTeachers(0, subject, exam, findAvalibleTeachers(subject, exam, MAIN_EXAMER_OF_ENG_SPEAKING))
            for i in range(1,4):
                if subject.room[i] == 'HALL' or subject.room[i][-1] == '1':
                    subject.teachers[i] = TA_DATA[i-1].name
                    findParentObj(TA_DATA, TA_DATA[i-1].name).totalTime += subject.timeLimit
            subject.teachers[subject.teachers.index('')] = 'AO'
            for i in range(subject.teachers.index(''),len(subject.room)):
                appendTeachers(i, subject, exam, findAvalibleTeachers(subject, exam, ORAL_EXAMER_OF_ENG_SPEAKING))
        elif 'istening' in subject.name and 'TSA' not in subject.name:
            appendTeachers(0, subject, exam, findAvalibleTeachers(subject, exam, MAIN_EXAMER_OF_ENG_LISTENING))
            appendTeachers(1, subject, exam, findAvalibleTeachers(subject, exam))
            for i in range(1,len(subject.room)):
                appendTA(i, subject)
        elif '說話' in subject.name or '説話' in subject.name:
            appendTeachers(0, subject, exam, findAvalibleTeachers(subject, exam, MAIN_EXAMER_OF_CHIN_SPEAKING))
            for i in range(1,3):
                subject.teachers[i] = TA_DATA[i-1].name
                findParentObj(TA_DATA, TA_DATA[i-1].name).totalTime += subject.timeLimit
            for i in range(3,len(subject.room)):
                appendTeachers(i, subject, exam, findAvalibleTeachers(subject, exam, ORAL_EXAMER_OF_CHIN_SPEAKING))
        elif '普通話' in subject.name:
            appendTeachers(0, subject, exam, findAvalibleTeachers(subject, exam, MAIN_EXAMER_OF_PTH))
            for i in range(1,len(subject.room)):
                appendTA(i, subject)
        elif '聆聽' in subject.name and 'TSA' not in subject.name and '普通話' not in subject.name:
            appendTeachers(0, subject, exam, findAvalibleTeachers(subject, exam, MAIN_EXAMER_OF_CHIN_LISTENING))
            for i in range(1,len(subject.room)):
                appendTA(i, subject)
        elif '視覺藝術' in subject.name:
            appendTeachers(0, subject, exam, findAvalibleTeachers(subject, exam, [MAIN_EXAMER_OF_VA[subject.form]]))

for exam in ET_DATA:
    for subject in list(filter(lambda x: '' in x.teachers, exam.subjects)):
        if 'HALL' in subject.room:
            for i in range(0,2):
                appendTeachers(i, subject, exam, findAvalibleTeachers(subject, exam))
            for i in range(2,len(subject.room)):
                appendTA(i, subject)
        else:
            for i in range(0,len(subject.room)):
                appendTeachers(i, subject, exam, findAvalibleTeachers(subject, exam))

Processing ...


Print out results

In [5]:
for exam in ET_DATA:
    for subject in exam.subjects:
        print('==================')
        print(exam.examDate)
        print(subject.name)
        print(subject.period)
        print(subject.room)
        print(subject.teachers)
        print('==================')

09/06/2023 (五)
中文 I
( 閱讀 )
8:30a.m.-9:45a.m.
['606', '607', '401', '402', '403']
['LWP', 'CLC', 'MHS', 'CTC', 'HLY']
09/06/2023 (五)
中文 II 
( 寫作 )
10:15a.m.-11:30a.m.
['606', '607', '401', '402', '403']
['LWF', 'LCP', 'NHH', 'NYK', 'OCW']
09/06/2023 (五)
中文II
( 寫作 )
8:30a.m.-10:45a.m.
['501', '502', '503', '504', '507']
['TSW', 'FCH', 'SWY', 'ASH', 'MKAY']
09/06/2023 (五)
中文 I
( 閱讀 )
11:15a.m.-12:45p.m
['501', '502', '503', '504', '507']
['LWP', 'CLC', 'MHS', 'NIF', 'FWC']
09/06/2023 (五)
English II
( Writing )
8:30a.m.-10:30a.m.
['HALL', 'HALL', 'HALL']
['LPS', 'NSW', 'LKHE']
09/06/2023 (五)
English I
( Reading )                       
11:00a.m.-12:30p.m.
['HALL', 'HALL', 'HALL']
['HLY', 'CWYU', 'LLS']
09/06/2023 (五)
中文III
( 聆聽及綜合 )
8:30a.m.-10:10a.m.
['HALL', 'HALL', 'HALL']
['LH', 'LKHE', 'LLS']
09/06/2023 (五)
English I
( Reading )                       
11:00a.m.-12:30p.m.
['HALL', 'HALL', 'HALL']
['LH', 'TSW', 'LYT']
09/06/2023 (五)
English III
( Listening )
10:45a.m.-1:00p.m.
['HALL', 

In [6]:
print('總上課和監考時間')
for teacher in TT_DATA:
    print(teacher.name + ': ' + str(teacher.totalTime) + ' mins')

總上課和監考時間
ASH: 880 mins
CHP: 840 mins
CHM: 875 mins
CMW: 860 mins
CTC: 895 mins
CHCH: 860 mins
CWYU: 940 mins
CLC: 860 mins
CLY: 860 mins
CFL: 865 mins
FWC: 895 mins
FCH: 890 mins
HWL: 965 mins
HCS: 850 mins
HLY: 965 mins
KSL: 905 mins
KYF: 895 mins
KWY: 860 mins
KHP: 910 mins
LWF: 930 mins
LKC: 840 mins
LWP: 835 mins
LSK: 840 mins
LCTA: 950 mins
LKHA: 910 mins
LPT: 840 mins
LPS: 870 mins
LCHO: 970 mins
LCF: 845 mins
LHY: 830 mins
LH: 890 mins
LSP: 925 mins
LSC: 870 mins
LYCI: 850 mins
LYL: 820 mins
LCP: 830 mins
MKAY: 855 mins
MHS: 895 mins
NHH: 840 mins
NIF: 870 mins
NWYI: 850 mins
NYK: 830 mins
NSW: 895 mins
OCW: 840 mins
SWY: 870 mins
SPM: 870 mins
SYF: 860 mins
TSW: 820 mins
TSY: 845 mins
TYF: 935 mins
WPH: 855 mins
WMS: 830 mins
WWY: 840 mins
WYM: 855 mins
WCS: 875 mins
YPC: 830 mins
YTHO: 900 mins


In [7]:
print('TA總監考時間')
for ta in TA_DATA:
    print(ta.name + ': ' + str(ta.totalTime) + ' mins')

TA總監考時間
TSYE: 735 mins
LYT: 640 mins
MLM: 645 mins
LLS: 680 mins
LKHE: 705 mins
LCHU: 710 mins


In [8]:
for teacher in TT_DATA:
    print(teacher.name + ': ' + str(teacher.exams))

ASH: {'19/06/2023 (一)': ['8:30a.m.-10:00a.m.'], '09/06/2023 (五)': ['8:30a.m.-10:45a.m.'], '12/06/2023 (一)': ['8:30a.m.-10:30a.m.'], '13/06/2023 (二)': ['11:15a.m.-12:30p.m.'], '14/06/2023 (三)': ['10:15a.m.-10:45a.m.', '11:00a.m.-12:30p.m.'], '16/06/2023 (五)': ['8:30a.m.-9:15a.m.', '11:15a.m.-12:30p.m.'], '20/06/2023 (二)': ['8:30a.m.-11:00a.m.']}
CHP: {'20/06/2023 (二)': ['10:15a.m.-10:40a.m.\n10:50a.m.-11:20a.m.'], '21/06/2023 (三)': ['10:15a.m.-10:45a.m.', '8:30a.m.-9:30a.m.'], '14/06/2023 (三)': ['8:30a.m.-9:45a.m.'], '15/06/2023 (四)': ['11:30a.m.-12:35p.m.'], '16/06/2023 (五)': ['10:15a.m.-12:30p.m.']}
CHM: {'12/06/2023 (一)': ['8:30a.m.-10:30a.m.'], '13/06/2023 (二)': ['8:30a.m.-10:45a.m.'], '14/06/2023 (三)': ['10:15a.m.-12:30p.m.'], '19/06/2023 (一)': ['10:30a.m.-11:25a.m.'], '20/06/2023 (二)': ['8:30a.m.-11:00a.m.']}
CMW: {'09/06/2023 (五)': ['10:45a.m.-1:00p.m.'], '15/06/2023 (四)': ['8:30a.m.-10:30a.m.', '11:00a.m.-11:45a.m.\n11:10a.m.-12:05p.m.'], '16/06/2023 (五)': ['10:30a.m.-11:05a.m.\

Output to xlxs

In [9]:
workbook = openpyxl.Workbook()
sheet = workbook.worksheets[0]
formDict = { 1 : '中一級', 2 : '中二級', 3 : '中三級', 4 : '中四級', 5 : '中五級', 6 : '中六級'}

orangeFill = PatternFill(patternType='solid', fgColor=Color(rgb='FFC000'))
yellowFill = PatternFill(patternType='solid', fgColor=Color(rgb='FFFF00'))
greyFill = PatternFill(patternType='solid', fgColor=Color(rgb='D9D9D9'))
cellborder = Border(left=Side(style='medium'), 
                     right=Side(style='medium'), 
                     top=Side(style='medium'), 
                     bottom=Side(style='medium'))


for i in range(1, ET_DATA[0].subjects[-1].form + 1):
    sheet.cell(row = sheet.max_row+2, column = 1).value = formDict[i]
    top = sheet.max_row+1
    for col, exam in enumerate(ET_DATA,start=1):
        sheet.cell(row = top, column = col).value = exam.examDate
        sheet.cell(row = top, column = col).border = cellborder
        sheet.cell(row = top, column = col).font = Font(bold=True)
        sheet.column_dimensions[get_column_letter(col)].width = 17
        current_row = top+1
        for subject in list(filter(lambda x: x.form == i, exam.subjects)):
            sheet.cell(row = current_row, column = col).value = subject.name
            sheet.cell(row = current_row, column = col).fill = orangeFill
            sheet.cell(row = current_row, column = col).font = Font(bold=True)

            sheet.cell(row = current_row+1, column = col).value = subject.timeLimit

            sheet.cell(row = current_row+2, column = col).value = subject.period

            for j in range(current_row, current_row+3):
                sheet.cell(row = j, column = col).border = cellborder
                sheet.cell(row = j, column = col).alignment = Alignment(horizontal='center', wrapText=True, vertical = 'center')

            current_row += 3
            for j in range(len(subject.room)):
                sheet.cell(row = current_row, column = col).value = subject.room[j] + ': ' + subject.teachers[j]
                sheet.cell(row = current_row, column = col).alignment = Alignment(horizontal='center', wrapText=True, vertical = 'center')
                sheet.cell(row = current_row, column = col).border = cellborder
                sheet.cell(row = current_row, column = col).fill = yellowFill
                current_row += 1
    
    for y in range(1, sheet.max_column+1):
        for x in range(top, sheet.max_row+1):
            if sheet.cell(row = x, column = y).value == None:
                sheet.cell(row = x, column = y).fill = greyFill


workbook.save('監考時間表.xlsx')